In [29]:
from ultralytics import YOLO
import cv2
import csv
import numpy as np
from sort.sort import *

# Your custom utility functions
from utilar import get_car, read_license_plate, write_csv

# Initialize variables
results = {}
mot_tracker = Sort()
coco_model = YOLO('yolov8m.pt')
license_plate_detector = YOLO('best.pt')

# Change this line to use the default camera
cap = cv2.VideoCapture('IMG.jpeg')

# Initialize CSV writer
csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame_nmr', 'car_id', 'car_bbox', 'license_bbox', 'license_text', 'license_bbox_score', 'license_text_score'])

# Main loop
vehicles = [2, 3, 5, 7]
frame_nmr = -1
ret = True

while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        #frame = cv2.resize(frame, (1280,720))
        results[frame_nmr] = {}
        detections = coco_model(frame)[0]
        detections_ = [[x1, y1, x2, y2, score] for x1, y1, x2, y2, score, class_id in detections.boxes.data.tolist() if int(class_id) in vehicles]
        track_ids = mot_tracker.update(np.array(detections_))
        license_plates = license_plate_detector(frame)[0]
        
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
            if car_id != -1:
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop)
                
                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    
                    # Write to CSV
                    csv_writer.writerow([frame_nmr, car_id, [xcar1, ycar1, xcar2, ycar2], [x1, y1, x2, y2], license_plate_text, score, license_plate_text_score])
                    
                    # Draw bounding boxes and text on the frame
                    cv2.rectangle(frame, (int(xcar1), int(ycar1)), (int(xcar2), int(ycar2)), (0, 255, 0), 2)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, license_plate_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Show the frame in a window
        cv2.imshow('Real-time License Plate Recognition', frame)
        # Press 'q' to quit
        if cv2.waitKey(100000000) & 0xFF == ord('q'):
            break

# Release video and close CSV
cap.release()
csv_file.close()
cv2.destroyAllWindows()


ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules

In [2]:
!pip3 install ultralytics.utils


ERROR: Could not find a version that satisfies the requirement ultralytics.utils (from versions: none)
ERROR: No matching distribution found for ultralytics.utils

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [28]:
from ultralytics import YOLO
import cv2
import csv
import numpy as np
from sort.sort import Sort

# Your custom utility functions
from utilar import get_car, read_license_plate, write_csv

# Initialize YOLO Models
coco_model = YOLO('yolov8n.pt')  # Vehicle Detection Model
license_plate_detector = YOLO('best.pt')  # Custom License Plate Model

# Initialize tracker
mot_tracker = Sort()

# Read the image
frame = cv2.imread('IMG.jpeg')

# Validate the image is loaded correctly
if frame is None:
    print("Error: Unable to load image.")
    exit()

# Open CSV file for writing results
csv_file = open('output.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame_nmr', 'car_id', 'car_bbox', 'license_bbox', 'license_text', 'license_bbox_score', 'license_text_score'])

# Object classes for vehicles (COCO dataset IDs)
vehicles = [2, 3, 5, 7]  # Car, Motorcycle, Bus, Truck
frame_nmr = 0  # Single frame processing

results = {}

# Vehicle Detection
vehicle_detections = coco_model(frame)[0]
vehicle_bboxes = [
    [x1, y1, x2, y2, score] for x1, y1, x2, y2, score, class_id in vehicle_detections.boxes.data.tolist()
    if int(class_id) in vehicles
]

# Tracking vehicles (Although tracking is unnecessary for a single frame)
track_ids = mot_tracker.update(np.array(vehicle_bboxes))

# License Plate Detection
license_plate_detections = license_plate_detector(frame)[0]

for license_plate in license_plate_detections.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = license_plate
    xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

    if car_id != -1:
        # Crop license plate from frame
        license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

        # Recognize Arabic license plate text
        license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop)

        if license_plate_text is not None:
            results[frame_nmr] = {
                'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                'license_plate': {
                    'bbox': [x1, y1, x2, y2],
                    'text': license_plate_text,
                    'bbox_score': score,
                    'text_score': license_plate_text_score
                }
            }

            # Write to CSV
            csv_writer.writerow([
                frame_nmr, car_id, [xcar1, ycar1, xcar2, ycar2],
                [x1, y1, x2, y2], license_plate_text, score, license_plate_text_score
            ])

            # Draw bounding boxes
            cv2.rectangle(frame, (int(xcar1), int(ycar1)), (int(xcar2), int(ycar2)), (0, 255, 0), 2)  # Vehicle
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # License Plate
            cv2.putText(frame, license_plate_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

# Show the processed image
cv2.imshow('License Plate Recognition', frame)
cv2.waitKey(0)  # Wait indefinitely until a key is pressed
cv2.destroyAllWindows()

# Cleanup
csv_file.close()


ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules

In [26]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # Load an official YOLO model to check if it works
model(frame)[0]

ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules

In [25]:
import sys
sys.modules['ultralytics.yolo.utils.callbacks.clearml'] = None

from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Load YOLO model
results = model.predict('IMG.jpeg', save=True, verbose=True)  # Run inference


ModuleNotFoundError: import of ultralytics.yolo.utils.callbacks.clearml halted; None in sys.modules